In [ ]:
import torch

In [ ]:
words=open("names.txt","r").read().splitlines()
b={}
for w in words:
    w='.'+w+'.'
    for ch1,ch2 in zip(w,w[1:]):
        pair=(ch1,ch2)
        b[pair]=b.get(pair,0)+1

In [ ]:
chars=sorted(list(set(''.join(words))))
chars=['.']+chars
stoi={s:i for i,s in enumerate(chars)}
itos={i:s for s,i in stoi.items()}

In [ ]:
N=torch.zeros((27,27),dtype=torch.int32)
for key in b.keys():
    ix1=stoi[key[0]]
    ix2=stoi[key[1]]
    N[ix1,ix2]=b[key]
N

In [ ]:
prob=N/N.sum(dim=1,keepdim=True)
g=torch.Generator().manual_seed(2147483647)
ix=torch.multinomial(prob[0],num_samples=1,replacement=True,generator=g).item()

In [ ]:
g=torch.Generator().manual_seed(2147483647)
for i in range(5):
    out=''
    ix=0
    while True:
        p=prob[ix]
        ix=torch.multinomial(p,1,True,generator=g).item()
        if not ix:
            break
        out+=itos[ix]
    print(out)
        


In [ ]:
import torch.nn.functional as F

In [ ]:
xs=[]
ys=[]
for w in words:
    chs='.'+w+'.'
    for ch1,ch2 in zip(chs,chs[1:]):
        ix1=stoi[ch1]
        ix2=stoi[ch2]
        print(ch1,ch2)
        xs.append(ix1)
        ys.append(ix2)
xs=torch.tensor(xs)
ys=torch.tensor(ys)


In [ ]:
xenc=F.one_hot(xs,num_classes=27).float()
xenc

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(xenc[0:50])

In [ ]:
g = torch.Generator().manual_seed(2147483647)
W = torch.randn((27, 27), generator=g, requires_grad=True)
lr=50
num=xs.nelement()


In [ ]:
for k in range(100):
    logits=xenc @ W
    fcount=logits.exp()
    P=fcount/fcount.sum(dim=1,keepdim=True)
    loss=-(P[torch.arange(num),ys]).log().mean()+0.01*(W**2).mean()  #L2正则损失
    print(loss)
    W.grad=None
    loss.backward()
    W.data-=lr*W.grad

In [ ]:
g = torch.Generator().manual_seed(2147483647)
for k in range(5):
    ix=0
    out=''
    while True:
        xen=F.one_hot(torch.tensor([ix]),num_classes=27).float()
        logi=xen @ W
        fc=logi.exp()
        p=fc/fc.sum()
        ix=torch.multinomial(p,1,replacement=True,generator=g).item()
        if not ix:
            break
        out+=itos[ix]
    print(out)